Build a convolutional neural network with TensorFlow.

In [1]:
from __future__ import print_function, division, absolute_import

import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

W0820 17:00:15.174103 139911792736128 deprecation.py:323] From <ipython-input-1-e418b69fb3ff>:7: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0820 17:00:15.176073 139911792736128 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0820 17:00:15.178272 139911792736128 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for 

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz


W0820 17:00:16.486287 139911792736128 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0820 17:00:16.490154 139911792736128 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz


W0820 17:00:16.943927 139911792736128 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: __init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [0]:
# training hyper-parameters
learning_rate = 0.001
num_steps = 500
batch_size = 128
display_step = 10

# Network hyperparameters
num_input = 784
num_classes = 10
dropout = 0.75

In [0]:
# tf graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

In [0]:
# create some wrappers
def conv2d(x, W, b, strides =1):
  #conv2d wrappers with bias and relu activations
  x = tf.nn.conv2d(x, W, strides =[1,strides, strides, 1], padding='SAME')
  x = tf.nn.bias_add(x, b)
  return tf.nn.relu(x)

def maxpool2d(x, k = 2):
  return tf.nn.max_pool(x, ksize = [1, k,k,1], strides = [1,k,k,1], padding='SAME')

In [0]:
# create model
def conv_net(x, weights, biases, dropout):
  # reshape input to [h,w,c]
  x = tf.reshape(x, shape = [-1, 28,28,1])
  
  # convolution layer
  conv1 = conv2d(x, weights['wc1'], biases['bc1'])
  # maxpooling
  conv1 = maxpool2d(conv1, k=2)
  
  # convolution layer
  conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
  # maxpooling
  conv2 = maxpool2d(conv2, k=2)
  
  # fully connected layer
  # reshape conv2 to fully connected layer
  fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
  
  fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
  fc1 = tf.nn.relu(fc1)
  #apply dropout
  
  fc1 = tf.nn.dropout(fc1, dropout)
  
  # output class prediciton
  out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
  return out

In [0]:
# store layer weights and biases
weights ={
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64,1024])),
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [7]:
# construct the model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

#define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

train_op = optimizer.minimize(loss_op)

#evaluate the model
correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

W0820 17:04:06.788908 139911792736128 deprecation.py:506] From <ipython-input-5-e0e8b66801e3>:23: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0820 17:04:06.807782 139911792736128 deprecation.py:323] From <ipython-input-7-07001f58a52f>:6: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
# start session
with tf.Session() as sess:
  
  sess.run(init)
  
  for step in range(1, num_steps+1):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    
    #run optimization op
    sess.run(train_op, feed_dict={X:batch_x, Y:batch_y, keep_prob:dropout})
    
    if step%display_step ==0 or step ==1 :
      #calculate batch loss and accuracy
      loss, acc = sess.run([loss_op, accuracy], feed_dict={X:batch_x,
                                                          Y:batch_y,
                                                          keep_prob:1.0})
      print("Step",str(step),"Minibatch loss",loss, "Accuracy",acc)
      
  print("Optimization finished !")
  
  # calculate accuracy for 256 MNIST test images
  print("Testing accuracy", sess.run(accuracy, feed_dict={X:mnist.test.images[:256],
                                                         Y:mnist.test.labels[:256],
                                                         keep_prob:1.0}))
  

Step 1 Minibatch loss 47784.402 Accuracy 0.125
Step 10 Minibatch loss 14690.845 Accuracy 0.4375
Step 20 Minibatch loss 7918.796 Accuracy 0.6796875
Step 30 Minibatch loss 5559.548 Accuracy 0.7265625
Step 40 Minibatch loss 3435.623 Accuracy 0.875
Step 50 Minibatch loss 4825.849 Accuracy 0.796875
Step 60 Minibatch loss 4120.708 Accuracy 0.8671875
Step 70 Minibatch loss 2456.6836 Accuracy 0.8671875
Step 80 Minibatch loss 2279.9583 Accuracy 0.90625
Step 90 Minibatch loss 1848.3865 Accuracy 0.8984375
Step 100 Minibatch loss 3356.0012 Accuracy 0.8671875
Step 110 Minibatch loss 1367.5732 Accuracy 0.921875
Step 120 Minibatch loss 1749.7465 Accuracy 0.890625
Step 130 Minibatch loss 1938.0431 Accuracy 0.9140625
Step 140 Minibatch loss 952.1836 Accuracy 0.921875
Step 150 Minibatch loss 1439.5458 Accuracy 0.921875
Step 160 Minibatch loss 643.26904 Accuracy 0.953125
Step 170 Minibatch loss 1052.5956 Accuracy 0.9453125
Step 180 Minibatch loss 1143.4999 Accuracy 0.921875
Step 190 Minibatch loss 1423.8